In [8]:
import numpy as np
from scipy.stats import bernoulli
from random import seed
from random import random

In [30]:
def initialize_dist(K):
    beta_distn_reward = []
    beta_distn_cost = []

    for i in range(K):
        beta_distn_reward.append([0.0,0.0])

    for i in range(k):
        beta_distn_cost.append([0.0,0.0])

    return beta_distn_reward, beta_distn_cost

def sample_reward_and_cost(arm_pulled, mu, cost):
    reward_received = bernoulli.rvs(mu[arm_pulled], size=1)[0]
    cost_received = bernoulli.rvs(cost[arm_pulled], size=1)[0]
    return reward_received, cost_received

def update_distn_and_budget(arm_pulled, budget, beta_distn_reward, beta_distn_cost, reward_received, cost_received):
    # cost_records.append(cost_received)
    # reward_records.append(reward_received)
    # arm_pulled_records.append(arm_pulled)
    beta_distn_reward[arm_pulled][0] += reward_received
    beta_distn_reward[arm_pulled][1] += (1-reward_received)
    beta_distn_cost[arm_pulled][0] += cost_received
    beta_distn_cost[arm_pulled][1] += (1-cost_received)
    return beta_distn_reward, beta_distn_cost

def choose_arm(beta_distn_reward, beta_distn_cost, k, budget, mu, cost):
    # reward_records = []
    # arm_pulled_records = []
    # cost_records = []

    sampled_mean_reward = np.array([0]*k, dtype=np.float)
    sampled_mean_cost = np.array([0]*k, dtype=np.float)

    if budget>0:
        for arm in range(k):
            sampled_mean_reward[arm] = np.random.beta(beta_distn_reward[arm][0]+1, beta_distn_reward[arm][1]+1)
            sampled_mean_cost[arm] = np.random.beta(beta_distn_cost[arm][0]+1, beta_distn_cost[arm][1]+1)
        arm_pulled = np.argmax(sampled_mean_reward/sampled_mean_cost)
        #reward_received, cost_received = sample_reward_and_cost(arm_pulled, mu, cost)
        #budget, beta_distn_reward, beta_distn_cost = update_distn_and_budget(arm_pulled, budget, beta_distn_reward, beta_distn_cost, reward_received,cost_received)
    return arm_pulled
    #return reward_received, cost_received, arm_pulled, budget, beta_distn_reward, beta_distn_cost

def compute_best_arm(mu, cost):
    return np.argmax(np.array(mu)/np.array(cost))

In [10]:
def initialize(K, B, min_mu_cost):
    """
    Parameters
    K - Number of arms
    B - Budget
    min_mu_cost - The minimum cost in the bernoulli trial
    Returns
    max_size - 2*B/min_cost
    N - A 2d array of size max_size and arms containing the total number of times an arm has been played at each time step
    X - A 2d array of size max_size and arms containing total reward obtained at each time step
    C - A 2d array of size max_size and arms containing total cost obtained at each time step
    arm_
    """
    max_size = int(2*B/min_mu_cost)
    N = np.zeros((max_size, K))
    X = np.zeros((max_size, K))
    C = np.zeros((max_size, K))
    arm_pulled = np.zeros(max_size, dtype = "int")
    return N,X,C,arm_pulled


def update(N, X, C, t):
    N[t] = N[t-1]
    X[t] = X[t-1]
    C[t] = C[t-1]

In [42]:
def fairness_with_budget_thompson_sampling(K, budget, mu, cost, alpha = None, R = None):
    
    min_mu_cost = min(cost)
    arm_pulled_count, reward_records, cost_records, arm_pulled_records = initialize(K, budget, min_mu_cost)

    t = 0
    beta_distn_reward, beta_distn_cost = initialize_dist(K)

    while budget > 0:
        t += 1
        play_thomp = True
        arm_pulled = -1
        if(alpha is not None and R is not None):
            unfair_arm = []
            unfair_val = []
            for i in range(K):
                if (R[i]*(t-1) - arm_pulled_count[t][i]) > alpha:
                    unfair_arm.append(i)
                    unfair_val.append(r[i]*(t-1) - arm_pulled_count[t][i])

            if unfair_arm:
                play_thomp = False
                arm_pulled = unfair_arm[np.argmax(np.array(unfair_val))]
                
        if play_thomp:
            arm_pulled = choose_arm(beta_distn_reward, beta_distn_cost, K, budget, mu, cost)
        
        update(arm_pulled_count, reward_records, cost_records, t)
        
        reward_received, cost_received = sample_reward_and_cost(arm_pulled, mu, cost)        
        beta_distn_reward, beta_distn_cost = update_distn_and_budget(arm_pulled, budget, beta_distn_reward, 
                                   beta_distn_cost, reward_received, cost_received)
        
        arm_pulled_count[t][arm_pulled] += 1
        budget -= cost_received
        cost_records[t][arm_pulled] += cost_received
        #print(cost_records[t][arm_pulled])
        reward_records[t][arm_pulled] += reward_received
        arm_pulled_records[t] = arm_pulled

    return arm_pulled_count, reward_records, cost_records, arm_pulled_records, t #N,

In [43]:
def compute_regret(C, arm_pulled, cost_means, reward_means, tB):
    """
    Returns 
    regret - an array size of tB+1 containing regret at each round
    #budget - total budget at end of round t
    """
    regret = np.zeros(tB+1)
    reg_sum = np.zeros(tB+1)
    budget_sum = np.zeros(tB+1)
    best_arm = compute_best_arm(reward_means, cost_means)
    #budget = np.zeros(tB+1)
    for i in range(tB+1):
        #budget[i] = np.sum(C[t])
        arm = arm_pulled[i]
        regret[i] = cost_means[arm]*(reward_means[best_arm]/cost_means[best_arm] - reward_means[arm]/cost_means[arm])
        #regret[i] = reward_means[best_arm] - reward_means[arm]
        reg_sum[i] = (reg_sum[i-1] + regret[i]) if i > 0 else regret[i]
        budget_sum[i] = np.sum(C[i])
        
    return regret, reg_sum, budget_sum

In [44]:
cost = [0.71138893, 0.74408936, 0.42790662, 0.81618107, 0.12610527,
       0.57237819, 0.83745534, 0.89507443, 0.92044633, 0.10855726]
mu = [0.53761232, 0.6889405 , 0.13180401, 0.52978279, 0.16823828,
       0.86604129, 0.99220976, 0.74377085, 0.34088901, 0.23490474]

In [41]:
j = 0
k = 10
B = 1000
while(j <= 9):
                 #budget
    # r = np.zeros(k)
    r = np.array([0.05]*k)          #fairness_array for different arms
    alpha = 1e10                       #tolerance_parameter for fairness

    # T = 1000
    #compute_regret(C, arm_pulled, cost_means, reward_means, tB):
    
    arm_pulled_count, reward_records, cost_records, arm_pulled_records, tB = fairness_with_budget_thompson_sampling(k, B, mu, cost)
    regret, reg_sum, budget_sum = compute_regret(cost_records, arm_pulled_records, cost, mu, tB)
    #return N,X,C,arm_pulled,t
    print(reg_sum[-1], tB)
#     regret_record = compute_regret(arm_pulled_records, mu, cost)
    j += 1
#     print (regret_record[-1])
#     print (len(regret_record))


0.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
2.0
1.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
2.0
0.0
0.0
0.0
0.0
2.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
3.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
3.0
2.0
2.0
1.0
2.0
2.0
2.0
2.0
3.0
3.0
4.0
1.0
3.0
3.0
3.0
3.0
2.0
3.0
3.0
3.0
3.0
4.0
3.0
3.0
3.0
3.0
3.0
3.0
4.0
4.0
5.0
4.0
4.0
4.0
4.0
5.0
6.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
7.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
6.0
6.0
6.0
8.0
6.0
6.0
6.0
2.0
9.0
6.0
4.0
6.0
6.0
6.0
6.0
6.0
5.0
6.0
6.0
6.0
6.0
6.0
7.0
6.0
6.0
4.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
3.0
6.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
10.0
3.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
11.0
11.0
11.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
1

160.0
160.0
6.0
160.0
160.0
160.0
160.0
160.0
160.0
160.0
160.0
14.0
160.0
160.0
160.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
161.0
6.0
161.0
161.0
161.0
161.0
161.0
162.0
162.0
162.0
163.0
163.0
163.0
163.0
163.0
163.0
164.0
164.0
165.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
6.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
166.0
167.0
167.0
167.0
167.0
167.0
167.0
168.0
168.0
168.0
168.0
168.0
168.0
168.0
168.0
168.0
168.0
168.0
169.0
169.0
169.0
169.0
169.0
169.0
170.0
170.0
171.0
172.0
172.0
7.0
172.0
172.0
172.0
172.0
172.0
173.0
173.0
174.0
174.0
174.0
175.0
8.0
175.0
175.0
176.0
176.0
176.0
176.0
176.0
177.0
177.0
177.0
177.0
177.0
177.0
177.0
177.0
177.0
177.0
177.0
177.0
177.0
178.0
178.0
178.0
179.0
179.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
181.0
181.0
181

355.0
355.0
355.0
355.0
355.0
355.0
355.0
355.0
355.0
355.0
355.0
355.0
355.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
356.0
357.0
357.0
357.0
357.0
357.0
357.0
357.0
357.0
357.0
357.0
357.0
358.0
358.0
358.0
358.0
358.0
358.0
358.0
358.0
358.0
358.0
358.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
12.0
360.0
360.0
360.0
360.0
360.0
360.0
360.0
360.0
361.0
361.0
361.0
361.0
361.0
361.0
361.0
15.0
361.0
362.0
362.0
362.0
362.0
362.0
362.0
362.0
362.0
362.0
363.0
363.0
363.0
363.0
363.0
363.0
363.0
363.0
363.0
363.0
363.0
363.0
364.0
364.0
364.0
7.0
364.0
364.0
364.0
16.0
364.0
364.0
364.0
364.0
364.0
364.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365.0
365

534.0
534.0
534.0
535.0
536.0
536.0
536.0
536.0
536.0
536.0
536.0
536.0
537.0
537.0
537.0
537.0
537.0
538.0
538.0
538.0
538.0
538.0
538.0
539.0
539.0
539.0
539.0
539.0
539.0
539.0
539.0
539.0
539.0
540.0
540.0
540.0
540.0
24.0
540.0
540.0
540.0
540.0
540.0
540.0
540.0
540.0
540.0
540.0
540.0
540.0
540.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
13.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
541.0
542.0
542.0
542.0
542.0
542.0
543.0
543.0
543.0
543.0
25.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
26.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
543.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
544.0
545.0
545.0
545.0
545.0
546.0
546.0
546.0
546.0
546.0
546.0
546.0
546.0
546.0
546.0
547.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
548.0
549.0
549.0
549.0
549.0
549.0
549.0
549.0
54

698.0
698.0
698.0
698.0
698.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
700.0
701.0
701.0
701.0
701.0
701.0
701.0
701.0
13.0
701.0
701.0
701.0
701.0
701.0
701.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
702.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
703.0
30.0
703.0
703.0
703.0
703.0
703.0
704.0
704.0
704.0
704.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
705.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
706.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0
707.0


...

174.61536986916275 8196
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
2.0
0.0
2.0
2.0
0.0
1.0
0.0
2.0
0.0
2.0
0.0
2.0
2.0
3.0
3.0
1.0
4.0
0.0
1.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
3.0
4.0
2.0
0.0
0.0
0.0
0.0
1.0
2.0
1.0
5.0
1.0
5.0
5.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
1.0
2.0
2.0
2.0
2.0
3.0
2.0
2.0
3.0
6.0
3.0
4.0
3.0
3.0
4.0
4.0
6.0
4.0
4.0
4.0
5.0
5.0
2.0
1.0
5.0
4.0
2.0
7.0
5.0
1.0
5.0
5.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
5.0
4.0
1.0
1.0
1.0
5.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
6.0
3.0
3.0
5.0
3.0
5.0
7.0
4.0
8.0
9.0
7.0
5.0
9.0
10.0
4.0
4.0
5.0
4.0
11.0
4.0
6.0
3.0
5.0
3.0
11.0
11.0
7.0
5.0
12.0
8.0
5.0
5.0
5.0
12.0
5.0
6.0
12.0
12.0
5.0
8.0
13.0
5.0
5.0
5.0
14.0
4.0
5.0
14.0
6.0
5.0
15.0
5.0
16.0
1.0
16.0
17.0
5.0
7.0
7.0
5.0
6.0
18.0
6.0
7.0
6.0
6.0
18.0
6.0
2.0
6.0
6.0
18.0
8.0
19.0
6.0
6.0
6.0
6.0
6.0
6.0
7.0
8.0
19.0
8.0
19.0
9.0
4.0
9.0
9.0
20.0
21.0
22.0
23.0
24.0
8.0
6.0
6.0
8.0
9.0
9.0
9.0
24.0
2.0
9.0
9.0
7.0
24.0
9.0
25.0
9.0
25.0
9.0
10

142.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
33.0
143.0
143.0
34.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
34.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
34.0
143.0
143.0
143.0
143.0
143.0
143.0
143.0
144.0
144.0
144.0
144.0
144.0
144.0
144.0
144.0
144.0
145.0
145.0
145.0
145.0
145.0
145.0
146.0
146.0
146.0
146.0
34.0
146.0
146.0
146.0
146.0
146.0
146.0
146.0
146.0
34.0
146.0
146.0
34.0
34.0
147.0
147.0
147.0
148.0
148.0
148.0
148.0
148.0
148.0
148.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
149.0
150.0
10.0
150.0
150.0
150.0
150.0
150.0
150.0
150.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
151.0
34.0
151.0
151.0
152.0
152.0
152.0
152.0
152.0
34.0
152.0
152.0
152.0
152.0
153.0
153.0
153.0
153.0
153.0
153.0
153.0
35.0
153.0
153.0
153.0
153.0
153.0
153.0
153.0
153.0
154.0
154.0
154.0
154.0
154.0
154.0
154.0
155.0
156.0
35.0
156.0
157.0

308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
308.0
309.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
310.0
311.0
311.0
311.0
311.0
312.0
312.0
312.0
312.0
312.0
312.0
312.0
313.0
313.0
313.0
314.0
314.0
315.0
316.0
316.0
316.0
316.0
316.0
316.0
316.0
317.0
318.0
318.0
318.0
318.0
318.0
318.0
318.0
318.0
318.0
318.0
318.0
319.0
319.0
320.0
320.0
320.0
320.0
320.0
320.0
320.0
320.0
320.0
320.0
321.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
322.0
323.0
323.0
323.0
323.0
323.0
324.0
324.0
324.0
324.0
324.0
325.0
325.0
325.0
325.0
325.0
40.0
325.0
325.0
325.0
325.0
325.0
325.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
326.0
41.0
326.0
326.0
326.0
326.0
326.0
327.0
327.0
327.0
327.0
327.0
327.0
327.0
327.0
327.0
327.0
327.0
327.0
41.0
327.0
3

513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
513.0
44.0
514.0
514.0
514.0
514.0
514.0
514.0
514.0
514.0
514.0
515.0
515.0
516.0
516.0
516.0
517.0
517.0
517.0
518.0
518.0
518.0
518.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
44.0
519.0
519.0
519.0
519.0
519.0
519.0
519.0
520.0
520.0
520.0
520.0
520.0
520.0
520.0
520.0
520.0
520.0
520.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
44.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
45.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
521.0
522.0
522.0
522.0
522.0
522.0
522.0
523.0
523.0
523.0
523.0
523.0
524.0
524.0
45.0
524.0
524.0
524.0
524.0
524.0
524.0
524.0
524.0
524.0
524.0
524.0
525.0
525.0
525.0
525.0
525.0
525.0
525.0
526.0
526.0
526.0
526.0
526.0
526.0
526.0
526.0
526.0
526.0
526.0
526.0
526.0
45.0
526.0
526.0
526.0
526.0
526.

684.0
684.0
684.0
684.0
685.0
685.0
685.0
685.0
685.0
685.0
685.0
685.0
685.0
685.0
685.0
685.0
685.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
686.0
687.0
687.0
687.0
687.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
47.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
688.0
689.0
689.0
689.0
689.0
689.0
690.0
690.0
690.0
690.0
690.0
690.0
690.0
691.0
692.0
692.0
692.0
692.0
692.0
693.0
693.0
48.0
693.0
694.0
695.0
695.0
695.0
695.0
696.0
696.0
697.0
698.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
699.0
48.0
700.0
700.0
700.0
700.0
700.0
700.0
700.0
701.0
701.0
701.0
701.0
701.0
701.0
701.0
701.0
701.0
702.0
703.0
703.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
704.0
7